#                                      Imputation to missing to the Relevant Data .....


In [199]:
import pyodbc
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import datetime
from scipy import stats
from   pandas import  DataFrame
from sklearn.model_selection import train_test_split



In [200]:
pd.set_option('display.max_columns', None)

In [201]:
#load the data
df_Acc = pd.read_csv ('../EDA1.csv', low_memory=False)


In [202]:
df_Acc.head()

,accident_day,accident_district,accident_district_hebrew,accident_hour,accident_hour_raw,accident_hour_raw_hebrew,accident_id,accident_minute,accident_month,accident_municipal_status,accident_municipal_status_hebrew,accident_natural_area,accident_natural_area_hebrew,accident_region,accident_region_hebrew,accident_severity,accident_severity_hebrew,accident_type,accident_type_hebrew,accident_year,accident_yishuv_name,accident_yishuv_shape,accident_yishuv_shape_hebrew,accident_yishuv_symbol,car_id,cross_direction,cross_direction_hebrew,cross_location,cross_location_hebrew,cross_mode,cross_mode_hebrew,day_in_week,day_in_week_hebrew,day_night,day_night_hebrew,day_type,day_type_hebrew,didnt_cross,didnt_cross_hebrew,driving_directions,driving_directions_hebrew,engine_volume,engine_volume_hebrew,file_type_police,geo_area,geo_area_hebrew,geom,id,km,km_accurate,km_raw,location_accuracy,location_accuracy_hebrew,manufacturing_year,multi_lane,multi_lane_hebrew,non_urban_intersection,non_urban_intersection_by_junction_number,non_urban_intersection_hebrew,object_distance,object_distance_hebrew,one_lane,one_lane_hebrew,police_unit,police_unit_hebrew,provider_and_id,provider_code,road1,road2,road_control,road_control_hebrew,road_intactness,road_intactness_hebrew,road_light,road_light_hebrew,road_object,road_object_hebrew,road_segment_from_km,road_segment_id,road_segment_length_km,road_segment_name,road_segment_number,road_segment_to_km,road_shape,road_shape_hebrew,road_sign,road_sign_hebrew,road_surface,road_surface_hebrew,road_type,road_type_hebrew,road_width,road_width_hebrew,seats,speed_limit,speed_limit_hebrew,street1,street1_hebrew,street2,street2_hebrew,total_weight,total_weight_hebrew,traffic_light,traffic_light_hebrew,vehicle_attribution,vehicle_attribution_hebrew,vehicle_damage,vehicle_damage_hebrew,vehicle_status,vehicle_status_hebrew,vehicle_type,vehicle_type_hebrew,weather,weather_hebrew
0,8,44,רחובות,0,1,00:00-00:59,2008000002,0,6,NaN,NaN,441.0,אזור רחובות,4.0,המרכז,3,קלה,1,פגיעה בהולך רגל,2008,רחובות,14.0,"100,000-199,999-תושב, יישוב יהודי",8400.0,1,9,לא ידוע,NaN,NaN,NaN,NaN,1,ראשון,5,לילה,2,ערב חג,9,לא ידוע,NaN,NaN,8,1801-2000,NaN,1,ישראל ללא יהודה ושומרון,0101000020E6100000B944974A2E684140A30D606F02E7...,8497204,NaN,NaN,NaN,1,עיגון מדויק,2001.0,NaN,NaN,NaN,NaN,NaN,9,לא ידוע,9.0,לא ידוע מס מסלולים,52,מרחב שפלה,32008000002,3,NaN,NaN,NaN,NaN,NaN,NaN,6,לילה לא ידוע,9,לא ידוע,NaN,NaN,NaN,NaN,NaN,NaN,9.0,אחר,5,לא ידוע,9,לא ידוע,2,עירונית לא בצומת,NaN,NaN,1.0,NaN,NaN,461.0,גורדון,NaN,NaN,2.0,2.0-2.9,9.0,לא ידוע,1.0,ישראלי,NaN,NaN,NaN,NaN,1,רכב נוסעים פרטי,9,לא ידוע
1,16,44,רחובות,0,1,00:00-00:59,2008000004,0,5,NaN,NaN,442.0,אזור ראשון לציון,4.0,המרכז,3,קלה,1,פגיעה בהולך רגל,2008,ראשון לציון,10.0,ראשון לציון,8300.0,1,9,לא ידוע,3.0,במעבר חצייה בלי רמזור,3.0,חצה רגיל,6,שישי,5,לילה,4,יום אחר,9,לא ידוע,1023.0,דרום,9,2001-3000,NaN,1,ישראל ללא יהודה ושומרון,0101000020E61000001DC15C7BAE63414001988C4940F9...,8197058,NaN,NaN,NaN,1,עיגון מדויק,2004.0,NaN,NaN,NaN,NaN,NaN,9,לא ידוע,1.0,חד סיטרי,52,מרחב שפלה,12008000004,1,NaN,NaN,7.0,אחר,NaN,NaN,6,לילה לא ידוע,9,לא ידוע,NaN,NaN,NaN,NaN,NaN,NaN,8.0,כביש ישר / צומת,5,לא ידוע,1,יבש,1,עירונית בצומת,1.0,עד 5 מטר,99.0,1.0,"עד 50 קמ""ש",149.0,היין,150.0,הסוכה,3.0,3.0-3.5,9.0,לא ידוע,1.0,ישראלי,NaN,NaN,NaN,NaN,2,משא עד 4 טון - אחוד (טרנזיט),1,בהיר
2,6,99,NaN,0,1,00:00-00:59,2008000006,0,5,99.0,מועצה מקומית,999.0,NaN,9.0,NaN,3,קלה,1,פגיעה בהולך רגל,2008,NaN,99.0,NaN,NaN,1,9,לא ידוע,NaN,NaN,NaN,NaN,3,שלישי,5,לילה,4,יום אחר,9,לא ידוע,NaN,NaN,0,לא ידוע,NaN,1,ישראל ללא יהודה ושומרון,0101000020E6100000D17AD0795A664140D6BEC83B06EB...,8177826,50.0,t,50.0,1,עיגון מדויק,NaN,NaN,NaN,NaN,NaN,NaN,9,לא ידוע,9.0,לא ידוע מס מסלולים,52,מרחב שפלה,12008000006,1,412.0,NaN,NaN,NaN,NaN,NaN,6,לילה לא ידוע,9,לא ידוע,4.0,4120020.0,1.5,מפגש עם מסילת ברזל - צומת נס ציונה,20.0,5.5,9.0,אחר,5,לא ידוע,9,לא ידוע,4,לא-עירונית לא בצומת,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,לא ידוע,1.0,ישראלי,NaN,NaN,NaN,NaN,17,אחר ולא ידוע,9,לא 

In [203]:
df_Acc.shape

(808779, 114)

## Step 1 : drop cols with more than 70 percent missing data

In [204]:
# find the variables with missing observations

cols_with_na = [var for var in df_Acc.columns if df_Acc[var].isnull().mean() > 0]
cols_with_na

['accident_district_hebrew',
 'accident_hour_raw_hebrew',
 'accident_municipal_status',
 'accident_municipal_status_hebrew',
 'accident_natural_area',
 'accident_natural_area_hebrew',
 'accident_region',
 'accident_region_hebrew',
 'accident_yishuv_name',
 'accident_yishuv_shape',
 'accident_yishuv_shape_hebrew',
 'accident_yishuv_symbol',
 'cross_location',
 'cross_location_hebrew',
 'cross_mode',
 'cross_mode_hebrew',
 'didnt_cross_hebrew',
 'driving_directions',
 'driving_directions_hebrew',
 'file_type_police',
 'geom',
 'km',
 'km_accurate',
 'km_raw',
 'manufacturing_year',
 'multi_lane',
 'multi_lane_hebrew',
 'non_urban_intersection',
 'non_urban_intersection_by_junction_number',
 'non_urban_intersection_hebrew',
 'object_distance_hebrew',
 'one_lane',
 'one_lane_hebrew',
 'road1',
 'road2',
 'road_control',
 'road_control_hebrew',
 'road_intactness',
 'road_intactness_hebrew',
 'road_segment_from_km',
 'road_segment_id',
 'road_segment_length_km',
 'road_segment_name',
 'road_

In [205]:
# features with a lot missing values
#df_Acc1 =df_Acc.copy()

mis_val = df_Acc.isnull().mean()



df = pd.DataFrame.from_dict(mis_val)
#print(mis_val)

df  = np.where(mis_val > 0.7)
#print(df)

coloNames = []
for i in df:
    coloNames.append (df_Acc.columns[i])
print(coloNames)
print(list(coloNames[0]))

[Index(['cross_location_hebrew', 'cross_mode_hebrew', 'file_type_police', 'km',
       'km_accurate', 'km_raw', 'multi_lane_hebrew', 'non_urban_intersection',
       'non_urban_intersection_by_junction_number',
       'non_urban_intersection_hebrew', 'road1', 'road2', 'road_control',
       'road_control_hebrew', 'road_segment_from_km', 'road_segment_id',
       'road_segment_length_km', 'road_segment_name', 'road_segment_number',
       'road_segment_to_km', 'street2', 'street2_hebrew',
       'vehicle_status_hebrew'],
      dtype='object')]
['cross_location_hebrew', 'cross_mode_hebrew', 'file_type_police', 'km', 'km_accurate', 'km_raw', 'multi_lane_hebrew', 'non_urban_intersection', 'non_urban_intersection_by_junction_number', 'non_urban_intersection_hebrew', 'road1', 'road2', 'road_control', 'road_control_hebrew', 'road_segment_from_km', 'road_segment_id', 'road_segment_length_km', 'road_segment_name', 'road_segment_number', 'road_segment_to_km', 'street2', 'street2_hebrew', 'vehicl

In [206]:
# for example
df_Acc["multi_lane_hebrew"].isnull().mean()
# too much missing values so all the above cols will be dropped

0.9123852127713504

In [207]:
df_Acc["multi_lane"].isnull().mean()



0.40845892388402766

In [208]:
df_Acc_org = df_Acc.copy()
df_Acc.shape

(808779, 114)

In [209]:
df_Acc.shape

(808779, 114)

In [210]:
for i , col in enumerate(coloNames[0]):
    if i > 0:
        
        df_Acc.drop([list(coloNames[0])[i]], axis= 1 ,inplace=True)

In [211]:
df_Acc.shape

(808779, 92)

In [214]:
df_Acc.head()

,accident_day,accident_district,accident_district_hebrew,accident_hour,accident_hour_raw,accident_hour_raw_hebrew,accident_id,accident_minute,accident_month,accident_municipal_status,accident_municipal_status_hebrew,accident_natural_area,accident_natural_area_hebrew,accident_region,accident_region_hebrew,accident_severity,accident_severity_hebrew,accident_type,accident_type_hebrew,accident_year,accident_yishuv_name,accident_yishuv_shape,accident_yishuv_shape_hebrew,accident_yishuv_symbol,car_id,cross_direction,cross_direction_hebrew,cross_location,cross_location_hebrew,cross_mode,day_in_week,day_in_week_hebrew,day_night,day_night_hebrew,day_type,day_type_hebrew,didnt_cross,didnt_cross_hebrew,driving_directions,driving_directions_hebrew,engine_volume,engine_volume_hebrew,geo_area,geo_area_hebrew,geom,id,location_accuracy,location_accuracy_hebrew,manufacturing_year,multi_lane,object_distance,object_distance_hebrew,one_lane,one_lane_hebrew,police_unit,police_unit_hebrew,provider_and_id,provider_code,road_intactness,road_intactness_hebrew,road_light,road_light_hebrew,road_object,road_object_hebrew,road_shape,road_shape_hebrew,road_sign,road_sign_hebrew,road_surface,road_surface_hebrew,road_type,road_type_hebrew,road_width,road_width_hebrew,seats,speed_limit,speed_limit_hebrew,street1,street1_hebrew,total_weight,total_weight_hebrew,traffic_light,traffic_light_hebrew,vehicle_attribution,vehicle_attribution_hebrew,vehicle_damage,vehicle_damage_hebrew,vehicle_status,vehicle_type,vehicle_type_hebrew,weather,weather_hebrew
0,8,44,רחובות,0,1,00:00-00:59,2008000002,0,6,NaN,NaN,441.0,אזור רחובות,4.0,המרכז,3,קלה,1,פגיעה בהולך רגל,2008,רחובות,14.0,"100,000-199,999-תושב, יישוב יהודי",8400.0,1,9,לא ידוע,NaN,NaN,NaN,1,ראשון,5,לילה,2,ערב חג,9,לא ידוע,NaN,NaN,8,1801-2000,1,ישראל ללא יהודה ושומרון,0101000020E6100000B944974A2E684140A30D606F02E7...,8497204,1,עיגון מדויק,2001.0,NaN,9,לא ידוע,9.0,לא ידוע מס מסלולים,52,מרחב שפלה,32008000002,3,NaN,NaN,6,לילה לא ידוע,9,לא ידוע,9.0,אחר,5,לא ידוע,9,לא ידוע,2,עירונית לא בצומת,NaN,NaN,1.0,NaN,NaN,461.0,גורדון,2.0,2.0-2.9,9.0,לא ידוע,1.0,ישראלי,NaN,NaN,NaN,1,רכב נוסעים פרטי,9,לא ידוע
1,16,44,רחובות,0,1,00:00-00:59,2008000004,0,5,NaN,NaN,442.0,אזור ראשון לציון,4.0,המרכז,3,קלה,1,פגיעה בהולך רגל,2008,ראשון לציון,10.0,ראשון לציון,8300.0,1,9,לא ידוע,3.0,במעבר חצייה בלי רמזור,3.0,6,שישי,5,לילה,4,יום אחר,9,לא ידוע,1023.0,דרום,9,2001-3000,1,ישראל ללא יהודה ושומרון,0101000020E61000001DC15C7BAE63414001988C4940F9...,8197058,1,עיגון מדויק,2004.0,NaN,9,לא ידוע,1.0,חד סיטרי,52,מרחב שפלה,12008000004,1,NaN,NaN,6,לילה לא ידוע,9,לא ידוע,8.0,כביש ישר / צומת,5,לא ידוע,1,יבש,1,עירונית בצומת,1.0,עד 5 מטר,99.0,1.0,"עד 50 קמ""ש",149.0,היין,3.0,3.0-3.5,9.0,לא ידוע,1.0,ישראלי,NaN,NaN,NaN,2,משא עד 4 טון - אחוד (טרנזיט),1,בהיר
2,6,99,NaN,0,1,00:00-00:59,2008000006,0,5,99.0,מועצה מקומית,999.0,NaN,9.0,NaN,3,קלה,1,פגיעה בהולך רגל,2008,NaN,99.0,NaN,NaN,1,9,לא ידוע,NaN,NaN,NaN,3,שלישי,5,לילה,4,יום אחר,9,לא ידוע,NaN,NaN,0,לא ידוע,1,ישראל ללא יהודה ושומרון,0101000020E6100000D17AD0795A664140D6BEC83B06EB...,8177826,1,עיגון מדויק,NaN,NaN,9,לא ידוע,9.0,לא ידוע מס מסלולים,52,מרחב שפלה,12008000006,1,NaN,NaN,6,לילה לא ידוע,9,לא ידוע,9.0,אחר,5,לא ידוע,9,לא ידוע,4,לא-עירונית לא בצומת,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,לא ידוע,1.0,ישראלי,NaN,NaN,NaN,17,אחר ולא ידוע,9,לא ידוע
3,1,43,רמלה,0,1,00:00-00:59,2008000007,0,7,NaN,NaN,431.0,אזור לוד,4.0,המרכז,3,קלה,1,פגיעה בהולך רגל,2008,לוד,15.0,"50,000-99,999-תושב, יישוב יהודי",7000.0,1,2,משמאל לימין,1.0,לא במעבר חצייה ליד צומת,1.0,3,שלישי,5,לילה,4,יום אחר,9,לא ידוע,1023.0,דרום,9,2001-3000,1,ישראל ללא יהודה ושומרון,0101000020E6100000AFC2606BDD7141409460941586F1...,8175406,1,עיגון מדויק,2001.0,NaN,9,לא ידוע,9.0,לא ידוע מס מסלולים,52,מרחב שפלה,12008000007,1,NaN,NaN,6,לילה לא ידוע,9,לא ידוע,8.0,כביש ישר / צומת,5,לא ידוע,1,יבש,1,עירונית בצומת,5.0,יותר מ- 14 מטר,4.0,1.0,"עד 50 קמ""ש",403.0,ז'בוטינסקי,2.0,2.0-2.9,9.0,לא ידוע,1.0,ישראלי,NaN,NaN,NaN,3,משא עד 4 טון - לא אחוד (טנדר),1,בהיר
4,4,43,רמלה,0,1,00:00-00:

In [215]:
df_Acc.shape

(808779, 92)

In [216]:

numeric_cols = df_Acc.select_dtypes(include=np.number).columns.tolist()


In [217]:
df_Acc.columns

Index(['accident_day', 'accident_district', 'accident_district_hebrew',
       'accident_hour', 'accident_hour_raw', 'accident_hour_raw_hebrew',
       'accident_id', 'accident_minute', 'accident_month',
       'accident_municipal_status', 'accident_municipal_status_hebrew',
       'accident_natural_area', 'accident_natural_area_hebrew',
       'accident_region', 'accident_region_hebrew', 'accident_severity',
       'accident_severity_hebrew', 'accident_type', 'accident_type_hebrew',
       'accident_year', 'accident_yishuv_name', 'accident_yishuv_shape',
       'accident_yishuv_shape_hebrew', 'accident_yishuv_symbol', 'car_id',
       'cross_direction', 'cross_direction_hebrew', 'cross_location',
       'cross_location_hebrew', 'cross_mode', 'day_in_week',
       'day_in_week_hebrew', 'day_night', 'day_night_hebrew', 'day_type',
       'day_type_hebrew', 'didnt_cross', 'didnt_cross_hebrew',
       'driving_directions', 'driving_directions_hebrew', 'engine_volume',
       'engine_volum

In [218]:
import seaborn as sns

def diagnostic_plots(df, variable):
    # function takes a dataframe (df) and
    # the variable of interest as arguments

    # define figure size
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.distplot(df[variable], bins=30)
    plt.title('Histogram')

    # boxplot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()

In [219]:
float_col = df_Acc.select_dtypes(include=['float64']) # This will select float columns only

print(float_col.columns.values)

# list(float_col.columns.values)
for col in float_col.columns.values:
    print(col)
    if col =='road_segment_from_km'  or col == 'road_segment_length_km'  or col == 'road_segment_to_km':
        continue
    else :
        df_Acc[col] = df_Acc[col].astype('Int64')
  

['accident_municipal_status' 'accident_natural_area' 'accident_region'
 'accident_yishuv_shape' 'accident_yishuv_symbol' 'cross_location'
 'cross_mode' 'driving_directions' 'manufacturing_year' 'multi_lane'
 'one_lane' 'road_intactness' 'road_shape' 'road_width' 'seats'
 'speed_limit' 'street1' 'total_weight' 'traffic_light'
 'vehicle_attribution' 'vehicle_damage' 'vehicle_status']
accident_municipal_status
accident_natural_area
accident_region
accident_yishuv_shape
accident_yishuv_symbol
cross_location
cross_mode
driving_directions
manufacturing_year
multi_lane
one_lane
road_intactness
road_shape
road_width
seats
speed_limit
street1
total_weight
traffic_light
vehicle_attribution
vehicle_damage
vehicle_status


In [220]:
print(df_Acc.info(verbose=True ,memory_usage='deep'), "\n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808779 entries, 0 to 808778
Data columns (total 92 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   accident_day                      808779 non-null  int64 
 1   accident_district                 808779 non-null  int64 
 2   accident_district_hebrew          702837 non-null  object
 3   accident_hour                     808779 non-null  int64 
 4   accident_hour_raw                 808779 non-null  int64 
 5   accident_hour_raw_hebrew          469360 non-null  object
 6   accident_id                       808779 non-null  int64 
 7   accident_minute                   808779 non-null  int64 
 8   accident_month                    808779 non-null  int64 
 9   accident_municipal_status         433941 non-null  Int64 
 10  accident_municipal_status_hebrew  433897 non-null  object
 11  accident_natural_area             747275 non-null  Int64 
 12  ac

In [221]:
df_Acc["driving_directions"]

0         <NA>
1         1023
2         <NA>
3         1023
4         <NA>
          ... 
808774       0
808775    4433
808776       0
808777       0
808778    4433
Name: driving_directions, Length: 808779, dtype: Int64

# drop the na rows of cols with less than 0.05 missing ...

In [222]:
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    #print(mis_val)
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    return mis_val_table_ren_columns


In [223]:
# features with a little missing values
df =df_Acc.copy()

df =missing_values_table(df)
df = pd.DataFrame.from_dict(df)
#print(df['% of Total Values'])

df  = np.where((df['% of Total Values'] < 0.05)  & (df['% of Total Values'] > 0))
print(df)
df_005_missing = pd.DataFrame()
coloNames = []
for i in df:
    coloNames.append (df_Acc.columns[i])
print(coloNames)
    
    

(array([37, 51, 64, 65, 74], dtype=int64),)
[Index(['didnt_cross_hebrew', 'object_distance_hebrew', 'road_shape',
       'road_shape_hebrew', 'seats'],
      dtype='object')]


In [224]:
df_Acc.dropna(subset = ['seats' , 'road_shape'], inplace=True)


In [225]:
df_Acc.shape

(808576, 92)

# Later 

In [155]:
# checking
# features with a lot missing values
#df_Acc1 =df_Acc.copy()

mis_val = df_Acc.isnull().mean()



df = pd.DataFrame.from_dict(mis_val)
#print(mis_val)

df  = np.where(mis_val > 0.4)
#print(df)

coloNames = []
for i in df:
    coloNames.append (df_Acc.columns[i])
#print(coloNames)
print(list(coloNames[0]))
#more 10 features .....
#['multi_lane', 'cross_mode', 'cross_location', 'accident_municipal_status', 'accident_municipal_status_hebrew', 'street1_hebrew', 'accident_hour_raw_hebrew', 'vehicle_status', 'vehicle_damage', 'vehicle_damage_hebrew']


['accident_hour_raw_hebrew', 'accident_municipal_status', 'accident_municipal_status_hebrew', 'cross_location', 'cross_location_hebrew', 'cross_mode', 'multi_lane', 'street1_hebrew', 'vehicle_damage', 'vehicle_damage_hebrew', 'vehicle_status']


In [156]:
df_Acc['accident_district_hebrew']

0          רחובות
1          רחובות
2             NaN
3            רמלה
4            רמלה
           ...   
808774    ירושלים
808775    תל אביב
808776        צפת
808777    תל אביב
808778    ירושלים
Name: accident_district_hebrew, Length: 808576, dtype: object

In [157]:

df_Acc['accident_district_null'] = np.where(df_Acc['accident_district_hebrew'].isnull(), 1, 0)
df_Acc['accident_district_null']

0         0
1         0
2         1
3         0
4         0
         ..
808774    0
808775    0
808776    0
808777    0
808778    0
Name: accident_district_null, Length: 808576, dtype: int32

In [158]:
df_Acc["driving_directions"].isnull().sum()
##to do One Hot Encoding


167753

In [159]:
df_Acc["driving_directions_hebrew"]

0                   NaN
1                  דרום
2                   NaN
3                  דרום
4                   NaN
              ...      
808774          לא ידוע
808775    \tממזרח למערב
808776          לא ידוע
808777          לא ידוע
808778    \tממזרח למערב
Name: driving_directions_hebrew, Length: 808576, dtype: object

In [198]:
# #one hot encoding


# from sklearn.preprocessing import OneHotEncoder


# #pd.get_dummies(df_Acc['accident_year'] , prefix="year", )
# #tmp = pd.get_dummies(df_Acc['accident_month'], prefix="month" ) # ,drop_first=True)

# tmpyear = pd.get_dummies(df_Acc['accident_year'] , prefix="year")
# tmpyear




In [197]:
# # and now for all variables together: train set

# tmpmonth = pd.get_dummies(df_Acc["accident_month"] , prefix="month")


# tmpmonth.head()

In [196]:
# df_Acc.shape

In [195]:

# tmptype = pd.get_dummies(df_Acc["accident_type"] , prefix="acc_type")


# tmptype.head()



In [191]:
# df_Acc.shape

In [192]:

# tmpday = pd.get_dummies(df_Acc["day_in_week"] , prefix="day")


# tmpday.head()



In [193]:

# tmpday = pd.get_dummies(df_Acc["day_night"] , prefix="daynight")


# tmpday.head()


In [194]:
# pd.concat([df_Acc['day_night'],
#            pd.get_dummies(df_Acc['day_night'] , prefix="day")], axis=1)


In [234]:
df_Acc_OHE = df_Acc.copy()

In [236]:
# now  split

# to split the datasets
from sklearn.model_selection import train_test_split

# for one hot encoding with sklearn
from sklearn.preprocessing import OneHotEncoder


In [90]:

# df_with_NA = df_Acc_Reduced[df_Acc_Reduced.columns.difference(cols)]
# print(df_with_NA.shape)
# numeric_cols = df_with_NA.select_dtypes(include=np.number).columns.tolist()

# df_with_NA[numeric_cols]
# df_with_NA[numeric_cols].shape


# df_with_NA[numeric_cols].isnull().mean()

# #numeric_cols
# #['accident_region',
# #  'accident_yishuv_shape',
# #  'accident_yishuv_symbol',
# #  'cross_location',
# #  'cross_mode',
# #  'latitude',
# #  'longitude',
# #  'one_lane',
# #  'road_intactness',
# #  'road_shape',
# #  'seats',
# #  'speed_limit',
# #  'street1',
# #  'traffic_light',
# #  'vehicle_attribution',
# #  'vehicle_damage',
# #  'vehicle_status']

In [237]:
# save step 2 of save the data
df_Acc.to_csv('../preimputation.csv', index=False)

In [238]:
df_Acc.shape

(808576, 92)

In [ ]:
#  Splitting ....

In [240]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    df_Acc[ df_Acc.columns.difference(["accident_severity","accident_severity_hebrew"])],  
    df_Acc[["accident_severity","accident_severity_hebrew"]],  
    test_size=0.3,  # percentage of obs in test set
    random_state=8)  # seed to ensure reproducibility

X_train.shape, X_test.shape ,y_test.shape ,y_train.shape

((566003, 90), (242573, 90), (242573, 2), (566003, 2))

In [241]:
X_test.to_csv('../X_test.csv', index=False)

In [242]:
y_test.to_csv('../y_test.csv', index=False)

In [243]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    X_train ,  
    y_train,  
    test_size=0.3,  # percentage of obs in test set
    random_state=10)  # seed to ensure reproducibility

X_train.shape, X_test.shape ,y_test.shape ,y_train.shape

((396202, 90), (169801, 90), (169801, 2), (396202, 2))

In [244]:
X_train.to_csv('../X_train.csv', index=False)

In [ ]:
y_train.to_csv('../y_train.csv', index=False)

In [ ]:
X_test.to_csv('../X_dev.csv', index=False)

In [ ]:
y_test.to_csv('../y_dev.csv', index=False)

#                

In [232]:
# split data and save it
df_Acc_Reduced.shape

(1443259, 85)

In [241]:
#df_Acc_Reduced[["accident_severity","accident_severity_hebrew"]]
#df_Acc_Reduced.columns

In [246]:
    #df_Acc_Reduced.columns.difference(["accident_severity","accident_severity_hebrew"])
# df_Acc_Reduced[["accident_severity","accident_severity_hebrew"]]

,accident_severity,accident_severity_hebrew
0,3,קלה
1,3,קלה
2,3,קלה
3,3,קלה
4,3,קלה
...,...,...
1443254,3,קלה
1443255,3,קלה
1443256,3,קלה
1443257,3,קלה


In [251]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    df_Acc_Reduced[ df_Acc_Reduced.columns.difference(["accident_severity","accident_severity_hebrew"])],  
    df_Acc_Reduced[["accident_severity","accident_severity_hebrew"]],  
    test_size=0.3,  # percentage of obs in test set
    random_state=8)  # seed to ensure reproducibility

X_train.shape, X_test.shape ,y_test.shape ,y_train.shape

((1010281, 83), (432978, 83), (432978, 2), (1010281, 2))

In [252]:
X_test.to_csv('../X_test.csv', index=False)

In [253]:
y_test.to_csv('../y_test.csv', index=False)

In [254]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    X_train ,  
    y_train,  
    test_size=0.3,  # percentage of obs in test set
    random_state=10)  # seed to ensure reproducibility

X_train.shape, X_test.shape ,y_test.shape ,y_train.shape

((707196, 83), (303085, 83), (303085, 2), (707196, 2))

In [255]:
X_train.to_csv('../X_train.csv', index=False)

In [256]:
y_train.to_csv('../y_train.csv', index=False)

In [257]:
X_test.to_csv('../X_dev.csv', index=False)

In [258]:
y_test.to_csv('../y_dev.csv', index=False)